# Analyze satellite imagery from Planet.com

## Imports

In [ ]:
%cd c:/Users/ekino/OneDrive - UW/SatMobFusion

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import numpy as np
import pandas as pd
import pyproj

import satmobfusion.convenience as c

import rasterio as rio
import rasterio.plot

In [ ]:
from config import init
init.run()

## Read-in config file containing information about the different locations/events

In [ ]:
locs = pd.read_csv("config/locations.csv", index_col="location")
locs

In [ ]:
# location = "Tulsa"
location = "Tulsa_small"
# location = "Champaign"

In [ ]:
folder = "data/satellite/"+location+"/"

## Read-in satellite images

In [ ]:
fn_pre = locs.loc[location, "fn_satellite_pre"]
fn_post = locs.loc[location, "fn_satellite_post"]

In [ ]:
src_pre = rio.open(folder+fn_pre)
src_post = rio.open(folder+fn_post)

In [ ]:
#print some metadata about the images
src = src_pre
# src = src_post

# src.profile
# src.meta
# src.crs
src.crs.to_proj4()
# src.bounds
# src.nodata
# src.count #number of bands

## Extract plotting window

In [ ]:
#window as given in locs
lon_min,lon_max,lat_min,lat_max = locs.loc[location, ["lon_min","lon_max","lat_min","lat_max"]]
print(lon_min,lon_max,lat_min,lat_max)

#convert the window to the CRS of the satellite image
inProj = pyproj.Proj(init='epsg:4326') #geog. CRS
# outProj = Proj(init='epsg:3857') #web mercator
outProj = pyproj.Proj(src.crs.to_proj4())
x_min,y_min = pyproj.transform(inProj,outProj,lon_min,lat_min)
x_max,y_max = pyproj.transform(inProj,outProj,lon_max,lat_max)
x_min,y_min,x_max,y_max

In [ ]:
window_pre = rio.windows.from_bounds(x_min,y_min,x_max,y_max, src_pre.transform)
window_post = rio.windows.from_bounds(x_min,y_min,x_max,y_max, src_post.transform)
window_pre,window_post

In [ ]:
window_bounds_pre = rasterio.windows.bounds(window_pre, src.transform)
window_bounds_post = rasterio.windows.bounds(window_post, src.transform)
print("Window bounds (pre): ", window_bounds_pre)
print("Window bounds (post): ", window_bounds_post)

#Define window extent (for us in imshow())
window_extent_pre = [window_bounds_pre[0], window_bounds_pre[2], window_bounds_pre[1], window_bounds_pre[3]]
window_extent_post = [window_bounds_post[0], window_bounds_post[2], window_bounds_post[1], window_bounds_post[3]]
print("Window extent (pre): ", window_extent_pre)
print("Window extent (post): ", window_extent_post)

In [ ]:
img_full_extent_pre = rio.plot.plotting_extent(src_pre)
img_full_extent_post = rio.plot.plotting_extent(src_post)
img_full_extent_pre,img_full_extent_post

## Read image from rasterio object

In [ ]:
img_pre = src_pre.read((1,2,3,4), window=window_pre, masked=True)
img_post = src_post.read((1,2,3,4), window=window_post, masked=True)

In [ ]:
img_pre
# img_post

In [ ]:
img_pre.shape

In [ ]:
src.width,src.height

In [ ]:
img_pre.dtype

## Derive RGB difference, NDVI, and grayscale images

In [ ]:
img_diff = img_post.astype("int16") - img_pre.astype("int16")

In [ ]:
img_diff

In [ ]:
#calculate grayscale image from RGB
img_pre_gray = 0.3*img_pre[0] + 0.59*img_pre[1] + 0.11*img_pre[2]
img_post_gray = 0.3*img_post[0] + 0.59*img_post[1] + 0.11*img_post[2]
img_diff_gray = img_post_gray - img_pre_gray

In [ ]:
#calculate NDVI for pre and post from NIR and Red bands
img_pre_ndvi = (img_pre[3] - img_pre[0]) / (img_pre[3] + img_pre[0])
img_post_ndvi = (img_post[3] - img_post[0]) / (img_post[3] + img_post[0])
img_diff_ndvi = img_post_ndvi - img_pre_ndvi

In [ ]:
img_pre_ndvi.shape

In [ ]:
img_pre.mean(),img_pre.min(),img_pre.max()

In [ ]:
# reshape images for plotting purposes
img_pre_vis_reshaped = np.dstack([img_pre[i] for i in range(3)])
img_post_vis_reshaped = np.dstack([img_post[i] for i in range(3)])
img_pre_vis_reshaped.shape

## Create maps

### Visual spectrum maps

In [ ]:
fig,ax = plt.subplots()
ax.imshow(img_pre_vis_reshaped, extent=window_extent_pre)
# ax.imshow(img_post_vis_reshaped, extent=window_extent_pre) ##extent=window_extent_pre will plot the same, just with the axes labeled with regards to the post image's coordinates (but the picture still aligns with pre)
# ax.imshow(img_diff_vis_reshaped, extent=window_extent_pre)

ax.add_artist(ScaleBar(1.0))

c.save_figure(fig, f"maps/{location}/"+"vis_.png", dpi=200)

### Gray maps

In [ ]:
fig,ax = plt.subplots()
# pos = ax.imshow(img_pre_gray, extent=window_extent_pre, cmap="gray", vmin=np.nanpercentile(np.ma.filled(img_pre_gray, np.nan), 1), vmax=np.nanpercentile(np.ma.filled(img_pre_gray, np.nan), 99))
# pos = ax.imshow(img_post_gray, extent=window_extent_pre, cmap="gray", vmin=np.nanpercentile(np.ma.filled(img_pre_gray, np.nan), 1), vmax=np.nanpercentile(np.ma.filled(img_pre_gray, np.nan), 99))
pos = ax.imshow(img_diff_gray, extent=window_extent_pre, cmap="gray", vmin=np.nanpercentile(np.ma.filled(img_diff_gray, np.nan), 1), vmax=np.nanpercentile(np.ma.filled(img_diff_gray, np.nan), 99))

ax.add_artist(ScaleBar(1.0))
fig.colorbar(pos, ax=ax, fraction=0.046, label="Luminosity", extend="both")

c.save_figure(fig, f"maps/{location}/"+"gray_.png", dpi=200)

### NDVI maps

In [ ]:
fig,ax = plt.subplots()
# pos = ax.imshow(img_pre_ndvi, extent=window_extent_pre, cmap="RdYlGn", vmin=np.nanpercentile(np.ma.filled(img_pre_ndvi, np.nan), 1), vmax=np.nanpercentile(np.ma.filled(img_pre_ndvi, np.nan), 99))
# pos = ax.imshow(img_post_ndvi, extent=window_extent_pre, cmap="RdYlGn", vmin=np.nanpercentile(np.ma.filled(img_pre_ndvi, np.nan), 1), vmax=np.nanpercentile(np.ma.filled(img_pre_ndvi, np.nan), 99))
pos = ax.imshow(img_diff_ndvi, extent=window_extent_pre, cmap="RdYlGn", vmin=np.nanpercentile(np.ma.filled(img_diff_ndvi, np.nan), 1), vmax=np.nanpercentile(np.ma.filled(img_diff_ndvi, np.nan), 99))
# ax.imshow(img_post_ndvi, extent=window_extent_pre)

# ax.imshow(r_reshaped)
ax.add_artist(ScaleBar(1.0))
fig.colorbar(pos, ax=ax, fraction=0.046, label="NDVI", extend="both")

c.save_figure(fig, f"maps/{location}/"+"NDVI_.png", dpi=200)

## Analyze differences

In [ ]:
img_diff.min()

### Histogram of change values

In [ ]:
fig,ax = plt.subplots()
# log = True
log = False
# ax.hist(img_diff.compressed(), bins=256, log=log)
# ax.hist(img_diff_gray.compressed(), bins=256, log=log)
ax.hist(img_diff_ndvi.compressed(), bins=256, log=log)
ax.axvline(0, color="black")
c.save_figure(fig, f"maps/{location}/"+"hist_diff.png", dpi=200)

In [ ]:
fig,axs = plt.subplots(1, 3, figsize=(16,12))

pos_red   = axs[0].imshow(img_diff[0], extent=window_extent_pre, cmap="BrBG")
pos_green = axs[1].imshow(img_diff[1], extent=window_extent_pre, cmap="BrBG")
pos_blue  = axs[2].imshow(img_diff[2], extent=window_extent_pre, cmap="BrBG")

axs[0].set_title("Reds", color="crimson")
axs[1].set_title("Greens", color="darkgreen")
axs[2].set_title("Blues", color="darkblue")

fig.colorbar(pos_red, ax=axs[0], fraction=0.032)
fig.colorbar(pos_green, ax=axs[1], fraction=0.032)
fig.colorbar(pos_blue, ax=axs[2], fraction=0.032)

fig.subplots_adjust(wspace=0.45)
c.save_figure(fig, f"maps/{location}/"+"diff_bands.png", dpi=200)